In [8]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

tem = ['peace.png', 'home.png', 'client.png']

import random

#  Felzenszwalb et al.
def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
 
	# initialize the list of picked indexes
	pick = []
 
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
 
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
		# loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
 
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
 
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
 
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
 
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
 
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
 
	# return only the bounding boxes that were picked
	return boxes[pick]

def component():
  return random.randint(0,255)

In [ ]:

img_rgb = cv2.imread('capture.png')



pos = [(170, 395), (170, 395+320), (170+1070, 395-320),(170+1070, 395), (170+1070, 395+320)]
ww = 315
hh = 188

for xx, yy in pos:
    img_sub = img_rgb[int(yy-10):int(yy+hh+10), int(xx-5):int(xx+ww+5)]
    game_sort = []

    for t in tem:

        #col = (component(),component(),component())
        template = cv2.imread(t)
        template = cv2.resize(template, (30,30))
        w, h = template.shape[0:2]

        res = cv2.matchTemplate(img_sub,template,cv2.TM_CCOEFF_NORMED)
        threshold = 0.5
        loc = np.where( res >= threshold)

        boundingbox = []
        for pt in zip(*loc[::-1]):
            boundingbox.append([pt[0],pt[1], pt[0]+w, pt[1]+h])
        # perform non-maximum suppression on the bounding boxes
        loc = non_max_suppression_slow(np.array(boundingbox), 0.5)
        for pt in loc:

            #cv2.rectangle(img_sub, (pt[0],pt[1]), (pt[2], pt[3]), col, 1)
            game_sort.append([t.split('.')[0], int(int(pt[0])/15), int(int(pt[1])/15)])
    game_sort = sorted(game_sort, key = lambda game_sort: (game_sort[1], game_sort[2]))
    print(len(game_sort))
    for iii in game_sort:
        print(iii)
    print('---------------------------------') 
    cv2.imwrite('res{0}-{1}.png'.format(xx,yy),img_sub)

289
['home', 0, 0]
['client', 0, 0]
['home', 0, 1]
['client', 0, 4]
['home', 0, 5]
['client', 0, 6]
['home', 0, 8]
['client', 0, 8]
['client', 0, 10]
['peace', 0, 11]
['home', 0, 11]
['client', 0, 11]
['peace', 1, 0]
['home', 1, 0]
['peace', 1, 1]
['home', 1, 1]
['peace', 1, 4]
['home', 1, 4]
['peace', 1, 6]
['peace', 1, 11]
['client', 2, 0]
['peace', 2, 2]
['peace', 2, 9]
['home', 2, 9]
['client', 2, 9]
['client', 2, 10]
['peace', 2, 11]
['home', 2, 11]
['client', 2, 11]
['client', 3, 9]
['home', 3, 11]
['client', 3, 11]
['home', 4, 0]
['client', 4, 0]
['peace', 4, 1]
['client', 4, 1]
['client', 4, 2]
['peace', 4, 3]
['home', 4, 3]
['client', 4, 4]
['peace', 4, 5]
['home', 4, 5]
['client', 4, 5]
['client', 4, 6]
['peace', 4, 7]
['home', 4, 7]
['client', 4, 7]
['client', 4, 8]
['peace', 4, 9]
['home', 4, 9]
['client', 4, 9]
['client', 4, 10]
['peace', 4, 11]
['peace', 4, 11]
['home', 4, 11]
['client', 4, 11]
['client', 5, 0]
['client', 5, 2]
['client', 5, 6]
['client', 5, 9]
['peace', 